### SuperTrend Date

In [ ]:
coin = 'JASMY/USD'
import ccxt
import schedule
import time
import pandas as pd
import numpy as np
import json
from datetime import datetime

pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

# ------------------------------------------------------------------ #

def get_keys(path):    
    with open(path) as f:
        return json.load(f)

# ------------------------------------------------------------------ #

CB_API_KEY = get_keys('/Users/kyledufrane/.secret/cbpro_key.json')
CB_API_SECRET = get_keys('/Users/kyledufrane/.secret/cbpro_secret.json')
CB_API_PASSPHRASE = get_keys('/Users/kyledufrane/.secret/cbpro_passphrase.json')

exchange = ccxt.coinbasepro({'apiKey': CB_API_KEY['key'],
                             'secret': CB_API_SECRET['secret'],
                             'password': CB_API_PASSPHRASE['passphrase']})

# bars = exchange.fetch_ohlcv(coin, timeframe='1m', limit=100)

# df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# ------------------------------------------------------------------ #

def true_range(dataframe):
    dataframe['previous_close'] = dataframe['close'].shift(1)
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-previous_close'] = abs(dataframe['high'] - dataframe['previous_close'])
    dataframe['low-previous_close'] = abs(dataframe['low'] - dataframe['previous_close'])
    true_range = dataframe[['high-low', 'high-previous_close', 'low-previous_close']].max(axis=1)  
    return true_range

# ------------------------------------------------------------------ #


def atr(dataframe, period=5):   
    dataframe['true_range'] = true_range(dataframe)   
#     print("Calculating average true range")   
    the_atr = dataframe['true_range'].rolling(period).mean()   
    return the_atr

# ------------------------------------------------------------------ #


def super_trend(dataframe, period=7, multiplier=3):
    hl2 = (dataframe['high'] + dataframe['low']) / 2
    dataframe['atr'] = atr(dataframe, period=period)
    dataframe['upperband'] = hl2 + (multiplier * dataframe['atr'])
    dataframe['lowerband'] = hl2 - (multiplier * dataframe['atr'])
    dataframe['in_uptrend'] = True
    
    for current in range(1, len(dataframe.index)):
        previous = current - 1
        
        if dataframe['close'][current] > dataframe['upperband'][previous]:
            dataframe['in_uptrend'][current]= True
        elif dataframe['close'][current] < dataframe['lowerband'][previous]:
            dataframe['in_uptrend'][current] = False
        else:
            dataframe['in_uptrend'][current] = dataframe['in_uptrend'][previous]
            if dataframe['in_uptrend'][current] and dataframe['lowerband'][current] < dataframe['lowerband'][previous]:
                dataframe['lowerband'][current] = dataframe['lowerband'][previous]
            if not dataframe['in_uptrend'][current] and dataframe['upperband'][current] > dataframe['upperband'][previous]:
                dataframe['upperband'][current] = dataframe['upperband'][previous]
    
    dataframe = dataframe.set_index('timestamp')
    return dataframe

# ------------------------------------------------------------------ #

in_position = False

# ------------------------------------------------------------------ #

def check_buy_sell_signal(dataframe):
    global in_position
    last_row = len(dataframe.index) - 1
    previous_row = last_row - 1
        
    if dataframe['in_uptrend'][last_row] != dataframe['in_uptrend'][previous_row]:
        print('****************** CHANGE IN STATE ******************')
        
    if not dataframe['in_uptrend'][previous_row] and dataframe['in_uptrend'][last_row]:
        print('Changing to uptrend, buy')
        if not in_position:
            order = exchange.create_market_buy_order(coin, 10)
            print(order)
            in_position=True
        else: 
            print("Already in position, nothing to do")
    
    if dataframe['in_uptrend'][previous_row] and not dataframe['in_uptrend'][last_row]:
        print('Changing to uptrend, sell')
        if in_position:
            order = exchange.create_market_sell_order(coin, 10)
            print(order)
            in_position=False
        else:
            print("You aren't in position, nothing to sell")
    return None

# ------------------------------------------------------------------ #            
            
def run_bot():
#     print(f"Fetching new bars for {datetime.now().isoformat()}....")
    bars = exchange.fetch_ohlcv(coin, timeframe='30s', limit=100)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    supertrend_data = super_trend(df)
    check_buy_sell_signal(supertrend_data)
    try:
        database = pd.read_csv('data/JASMY-USD.csv')
        database = database.set_index('timestamp')
        database = pd.concat([database, supertrend_data], axis=0)
        database.to_csv('data/JASMY-USD.csv', encoding='utf-8')
    except:
        supertrend_data.to_csv('data/JASMY-USD.csv', encoding='utf-8')

                           

schedule.every(4).seconds.do(run_bot)

while(True):
    schedule.run_pending()
    time.sleep(1)